In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from scipy.stats.mstats import trimmed_var
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

In [3]:
df = pd.read_csv("Data.csv")

In [4]:
df.head()

,Country Name,Country Code,Series Name,Series Code,2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Afghanistan,AFG,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,32.6,33.8,34.9,36.1,..
1,Afghanistan,AFG,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,15.6,16.4,17.4,18.5,..
2,Afghanistan,AFG,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,83.2,83.8,84.5,85,..
3,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,97.7,97.7,97.7,85.3,..
4,Afghanistan,AFG,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,97.1,97.1,97.1,81.7,..


In [5]:
unique_cols = df['Series Name'].nunique()
unique_cols

1492

In [6]:
data_dict = df[['Series Code', 'Series Name']][:1492]
data_dict.to_csv('data_dict.csv', index=True) 

In [7]:
df["2019 [YR2019]"]=df["2019 [YR2019]"].replace("..", None)
df["2019 [YR2019]"] = df["2019 [YR2019]"].astype(float)

In [8]:
df_trans=pd.pivot_table(df, values="2019 [YR2019]", index="Country Name", columns="Series Code", sort=False)

In [9]:
# Remove last 48 rows as they correspond to enties like 'world' and not individual countries
df_trans=df_trans.iloc[:-48]

In [10]:
df_trans.head()

Series Code,EG.CFT.ACCS.ZS,EG.CFT.ACCS.RU.ZS,EG.CFT.ACCS.UR.ZS,EG.ELC.ACCS.ZS,EG.ELC.ACCS.RU.ZS,EG.ELC.ACCS.UR.ZS,NY.ADJ.NNTY.CD,NY.ADJ.NNTY.PC.CD,NY.ADJ.DCO2.GN.ZS,NY.ADJ.DCO2.CD,...,SG.VAW.ARGU.ZS,SG.VAW.BURN.ZS,SG.VAW.GOES.ZS,SG.VAW.NEGL.ZS,SG.VAW.REFU.ZS,SH.SGR.PROC.P5,DT.NFL.UNEC.CD,DT.NFL.UNEP.CD,DT.NFL.UNTA.CD,DT.NFL.UNWT.CD
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,32.6,15.6,83.2,97.7,97.1,99.5,1.773194e+10,469.477760,1.290995,2.479358e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,82.0,64.7,93.4,100.0,100.0,100.0,1.244998e+10,4361.999293,1.295783,1.969622e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,99.7,98.8,99.9,99.5,98.7,99.8,1.341336e+11,3140.907179,4.167493,6.983390e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.449396e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,100.0,100.0,100.0,100.0,100.0,100.0,NaN,NaN,0.588906,2.038946e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_trans.shape

(217, 1329)

In [12]:
drop_threshold = round(0.4*len(df_trans)) # we will drop colums that have more than 40% missing values
drop_threshold

87

In [13]:
drop_cols = []
for col in df_trans.columns:
    if df_trans[col].isnull().sum()>drop_threshold:
        drop_cols.append(col)
len(drop_cols)

498

In [14]:
df_trans.drop(columns=drop_cols, axis=1, inplace=True)
df_trans.head()

Series Code,EG.CFT.ACCS.ZS,EG.CFT.ACCS.RU.ZS,EG.CFT.ACCS.UR.ZS,EG.ELC.ACCS.ZS,EG.ELC.ACCS.RU.ZS,EG.ELC.ACCS.UR.ZS,NY.ADJ.NNTY.CD,NY.ADJ.NNTY.PC.CD,NY.ADJ.DCO2.GN.ZS,NY.ADJ.DCO2.CD,...,GC.TAX.GSRV.VA.ZS,GC.TAX.GSRV.CN,GC.TAX.YPKG.RV.ZS,GC.TAX.YPKG.ZS,GC.TAX.YPKG.CN,NY.TTF.GNFS.KN,NE.TRD.GNFS.ZS,SL.UEM.TOTL.FE.NE.ZS,SL.UEM.TOTL.MA.NE.ZS,SL.UEM.TOTL.NE.ZS
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,32.6,15.6,83.2,97.7,97.1,99.5,1.773194e+10,469.477760,1.290995,2.479358e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,82.0,64.7,93.4,100.0,100.0,100.0,1.244998e+10,4361.999293,1.295783,1.969622e+08,...,18.859746,2.202572e+11,18.374484,26.006401,7.983584e+10,-1.144614e+10,76.279195,11.316,11.585,11.466
Algeria,99.7,98.8,99.9,99.5,98.7,99.8,1.341336e+11,3140.907179,4.167493,6.983390e+09,...,NaN,NaN,NaN,NaN,NaN,0.000000e+00,46.506843,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.449396e+07,...,NaN,NaN,NaN,NaN,NaN,-3.990228e+07,156.568779,NaN,NaN,NaN
Andorra,100.0,100.0,100.0,100.0,100.0,100.0,NaN,NaN,0.588906,2.038946e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_trans.isnull().sum(axis=1).sort_values().tail(10)

Country Name
American Samoa              580
Monaco                      588
Turks and Caicos Islands    591
Faroe Islands               592
Channel Islands             645
Northern Mariana Islands    652
British Virgin Islands      653
Isle of Man                 655
Gibraltar                   660
St. Martin (French part)    719
dtype: int64

In [16]:
drop_threshold = round(0.32*df_trans.shape[1]) # we will drop rows that have more than 60% missing values
drop_threshold

266

In [17]:
drop_rows = []
for row in df_trans.index:
    if df_trans.loc[row].isnull().sum()>drop_threshold:
        drop_rows.append(row)
len(drop_rows)

39

In [18]:
df_trans.drop(drop_rows, axis=0, inplace=True)


In [19]:
df_trans.isnull().sum(axis=1).sort_values().tail(10)

Country Name
Sao Tome and Principe    187
Hong Kong SAR, China     210
Barbados                 219
Cuba                     220
Marshall Islands         221
Liberia                  227
Antigua and Barbuda      227
Grenada                  253
Macao SAR, China         253
Palau                    256
dtype: int64

In [20]:
df_trans.shape

(178, 831)

#Multicolinearity Analysis



In [21]:
# # Create correlation matrix
# correlation = df_trans.select_dtypes("number").corr().abs()
# correlation
# # Plot heatmap of `correlation`
# sns.heatmap(correlation);

In [22]:
# # Drop features with high correlation
# threshold = 0.9
# # Select upper triangle of correlation matrix
# upper = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(bool))

# # Find features with correlation greater than threshold
# to_drop = [column for column in upper.columns if any(upper[column] >= threshold)]

In [23]:
# len(to_drop)

In [24]:
# df_trans = df_trans.drop(to_drop, axis=1)

In [25]:
full_keys=df[df["Country Name"]=="Afghanistan"][["Series Code","Series Name"]]

In [26]:
clean_dict=pd.DataFrame()
# for col in df_trans.columns:
clean_dict["Series Code"]=[col for col in df_trans.columns]
clean_dict["Series Name"]=[full_keys[full_keys['Series Code']==col]['Series Name'].values[0] for col in df_trans.columns]

In [27]:
clean_dict

,Series Code,Series Name
0,EG.CFT.ACCS.ZS,Access to clean fuels and technologies for coo...
1,EG.CFT.ACCS.RU.ZS,Access to clean fuels and technologies for coo...
2,EG.CFT.ACCS.UR.ZS,Access to clean fuels and technologies for coo...
3,EG.ELC.ACCS.ZS,Access to electricity (% of population)
4,EG.ELC.ACCS.RU.ZS,"Access to electricity, rural (% of rural popul..."
...,...,...
826,NY.TTF.GNFS.KN,Terms of trade adjustment (constant LCU)
827,NE.TRD.GNFS.ZS,Trade (% of GDP)
828,SL.UEM.TOTL.FE.NE.ZS,"Unemployment, female (% of female labor force)..."
829,SL.UEM.TOTL.MA.NE.ZS,"Unemployment, male (% of male labor force) (na..."


In [28]:
clean_dict.to_csv('data_dict.csv', index=True) 

In [29]:
df_trans.shape

(178, 831)

In [30]:
max(df_trans.isnull().sum())

60

In [31]:
df_trans_fill = df_trans.fillna(df_trans.median())

In [32]:
df_trans_fill.mean()

Series Code
EG.CFT.ACCS.ZS          6.864803e+01
EG.CFT.ACCS.RU.ZS       6.023596e+01
EG.CFT.ACCS.UR.ZS       7.721264e+01
EG.ELC.ACCS.ZS          8.504888e+01
EG.ELC.ACCS.RU.ZS       8.163427e+01
                            ...     
NY.TTF.GNFS.KN         -5.893521e+12
NE.TRD.GNFS.ZS          8.898768e+01
SL.UEM.TOTL.FE.NE.ZS    7.219309e+00
SL.UEM.TOTL.MA.NE.ZS    5.895253e+00
SL.UEM.TOTL.NE.ZS       6.355376e+00
Length: 831, dtype: float64

In [33]:
df_trans_fill.to_csv('clean_database.csv',index=True)